
## Calculate geometric eigenmodes

To calculate the geometric eigenmodes of a brain surface using the method outlined in [Pang *et al.* 
 (2023)](https://doi.org/10.1038/s41586-023-06098-1), we will first initialize the solver with a
 path to a surface file. Since `nsbtools` provides triangular surface meshes for human, macaque, and 
 marmoset cortices, we can provide these as follows:

In [1]:
import numpy as np
import importlib
from nsbtools.eigen import EigenSolver
from nsbtools.plotting import plot_brain
from nsbtools.utils import unmask

# Get path to human left hemisphere surface from nsbtools data
data_dir = importlib.resources.files('nsbtools.data')
lh_surf = data_dir / 'sp-human_tpl-fsLR_den-32k_hemi-L_midthickness.surf.gii'

# Initialize solver with surface
solver = EigenSolver(lh_surf)

Next, we can calculate the eigenmodes by solving the eigenvalue problem of the Laplace-Beltrami
operator on the surface mesh:

In [2]:
solver.solve()

Solver: spsolve (LU decomposition) ...



By default, the solver computes the first 100 eigenmodes and corresponding eigenvalues, which are  
 then stored in the `emodes` and `evals` attributes. We can visualize the first few modes via the 
 `plotting` module:

In [ ]:
# Get the first 3 eigenmodes
modes = solver.emodes[:, :3]

# Visualize on the cortical surface
plot_brain(lh_surf, modes, labels=['Mode 1', 'Mode 2', 'Mode 3'], cmap='seismic')


We can see that the modes have been calculated, but incorrectly included the medial wall. To exclude
 the medial wall, we can either use a surface file that has the medial wall removed, or provide a 
 boolean mask when initializing the solver. The same mask is then provided to our `unmask` function 
 to reintroduce the medial wall for plotting. We can try this now with a marmoset surface:


In [ ]:

# Load medial wall mask
marmo_medmask = np.load(data_dir / 'sp-marmoset_space-fsLR_den-10k_hemi-L_desc-medmask.npy').astype(bool)

# Initialize solver with marmoset surface and mask
marmo_lh_surf = data_dir / 'sp-marmoset_tpl-fsLR_den-10k_hemi-L_midthickness.surf.gii'
marmosolver = EigenSolver(marmo_lh_surf, medmask=marmo_medmask)

# Get the first 3 eigenmodes
marmosolver.solve()
marmodes = marmosolver.emodes[:, :3]

# Visualize on the cortical surface
marmodes_unmasked = unmask(marmodes, marmo_medmask)
plot_brain(marmo_lh_surf, marmodes_unmasked, labels=['Mode 1', 'Mode 2', 'Mode 3'], cmap='seismic')